# Polymer End of Life Approximation

This notebook very roughly approximates the volume of polymers reaching end of life in each region within the [Global Plastics AI Policy Tool](https://global-plastics-tool.org/) per year. It does this by distributing waste in each fate proportionally to the size of each sector at the time of waste generation. Having attributed waste by sector, a static matrix converts to polymers which are then summed across sectors.

Note that this is a highly approximate estimation. This notebook discusses limitations of this approach and alternatives after generating the output CSV file. See discussion below.

Uses data from [10.1126/science.adr3837](https://www.science.org/doi/10.1126/science.adr3837): A. Samuel Pottinger et al. Pathways to reduce global plastic waste mismanagement and greenhouse gas emissions by 2050. Science 386, 1168-1173 (2024).

<br>

## Setup

We first download and prepare to process the data.

In [1]:
import csv
import itertools

import requests
import toolz.itertoolz

### Constants

This script iterates across regions, fates, polymers, sectors, and simulation years. As a convienence, these are provided as constants after importing the required libraries along with the service URL.

In [2]:
REGIONS = ['china', 'eu30', 'na', 'mw']

FATES = [
    'eolLandfillMT',
    'eolIncinerationMT',
    'eolMismanagedMT',
    'eolRecyclingMT'
]

POLYMERS = [
    'ldpe',
    'hdpe',
    'pp',
    'ps',
    'pvc',
    'pet',
    'pur',
    'other thermoplastics',
    'other thermosets'
]

SECTORS = [
    'consumptionTransportationMT',
    'consumptionPackagingMT',
    'consumptionConstructionMT',
    'consumptionElectronicMT',
    'consumptionHouseholdLeisureSportsMT',
    'consumptionAgricultureMT',
    'consumptionOtherMT'
]


YEARS = list(range(2011, 2051))

SERVICE_FILES_URL = 'https://global-plastics-tool.org/data/%s'

We provide these values as Majority World (MW) instead of Rest of World (ROW) and North America (NA) instead of NAFTA. These labels encompass the same parts of the world in either labeling convention. That said, we encourage use of MW and NA. Alternative data exports with "legacy" names can be found in [the tool data directory](https://global-plastics-tool.org/data).

In [3]:
LEGACY_REGION_MAP = {
    'china': 'china',
    'eu30': 'eu30',
    'nafta': 'na',
    'row': 'mw'
}

In [4]:
def normalize_region(target):
    target_lower = target.strip().lower()
    return LEGACY_REGION_MAP.get(target_lower, target_lower)

### Download

Download the required data: simulation outputs and the static polymer matrix.

In [5]:
def download_file(filename):
    url = SERVICE_FILES_URL % filename
    download = requests.get(url)
    content = download.content.decode('utf-8')
    reader = csv.DictReader(content.splitlines())
    return list(reader)

In [6]:
sim_output_rows = download_file('web.csv')

In [7]:
polymer_rows = download_file('static_polymer_matrix.csv')

<br>

## Index Data

Next, we index the downloaded data such that we can pull individual values for the approximation equation.

### Keying functions

We look up values from dictionaries. To support that operation, these functions define the keys in which we index into those dictionaries.

In [8]:
def get_year_region_key(region, year):
    pieces = [region, year]
    pieces_str = map(lambda x: str(x).lower(), pieces)
    return '\t'.join(pieces_str)

In [9]:
def get_sector_key(region, year):
    pieces = [region, year]
    pieces_str = map(lambda x: str(x).lower(), pieces)
    return '\t'.join(pieces_str)

### Index fate propensities

With these functions in place, we next determine the waste fate ratios which is the percent of waste in a region / year going to a specific fate like recycling. For example the recycling ratio for EU30 in 2035.

In [10]:
fate_propensities = {}

for row in sim_output_rows:
    year = row['year']

    region = normalize_region(row['region'])
    
    fate_cols = FATES
    total = sum(map(lambda x: float(row[x]), fate_cols))

    individual_fates = map(lambda x: {'fate': x}, FATES)

    fates_with_keys = map(lambda x: {
        'fate': x['fate'],
        'key': x['fate']
    }, individual_fates)

    fates_with_vals = map(lambda x: {
        'fate': x['fate'],
        'propensity': float(row[x['key']]) / total
    }, fates_with_keys)

    fates_with_vals_flat = map(lambda x: (
        x['fate'],
        x['propensity']
    ), fates_with_vals)

    key = get_year_region_key(region, year)
    fate_propensities[key] = dict(fates_with_vals_flat)

In [11]:
def get_fate_propensity(region, year, fate):
    return fate_propensities[get_year_region_key(region, year)][fate]

### Index sector propensities

Afterwards, we determine how much each sector makes up of the total consumption for a region / year. For example, this would include what percent of China consumption from 2050 is from transportation.

In [12]:
sector_propensities = {}

for row in sim_output_rows:
    year = row['year']
    region = normalize_region(row['region'])

    sector_cols = SECTORS
    total = sum(map(lambda x: float(row[x]), sector_cols))

    individual_sectors = map(lambda x: {'sector': x}, SECTORS)

    sectors_with_keys = map(lambda x: {
        'sector': x['sector'],
        'key': x['sector']
    }, individual_sectors)

    sectors_with_vals = map(lambda x: {
        'sector': x['sector'],
        'propensity': float(row[x['key']]) / total
    }, sectors_with_keys)

    sectors_with_vals_flat = map(lambda x: (
        x['sector'],
        x['propensity']
    ), sectors_with_vals)

    key = get_year_region_key(region, year)
    sector_propensities[key] = dict(sectors_with_vals_flat)

In [13]:
def get_sector_propensity(region, year, sector):
    return sector_propensities[get_year_region_key(region, year)][sector]

### Index total waste

Additionally, we determine how much total waste is produced each year in each region across all fates. For example, the total waste generated in North Amercia in 2040.

In [14]:
total_waste = {}
    
for row in sim_output_rows:
    year = row['year']
    region = normalize_region(row['region'])
    
    fate_cols = FATES
    total = sum(map(lambda x: float(row[x]), fate_cols))

    key = get_year_region_key(region, year)
    total_waste[key] = total

In [15]:
def get_total_waste(region, year):
    return total_waste[get_year_region_key(region, year)]

### Index sector polymers

Finally, we determine how much each polymer makes up each sector within a region. For example, what percent of packaging is polystyrene in EU30 by mass. This is a static matrix so does not change year to year in the simulation.

In [16]:
def parse_polymer_percent(target):
    return float(target.replace('%', '')) / 100

In [17]:
sector_polymers = {}
    
for row in polymer_rows:
    region = normalize_region(row['region'])
    sector = row['sector']

    polymers_propensities_unnorm = dict(map(
        lambda x: (x, parse_polymer_percent(row[x])),
        POLYMERS
    ))

    total = sum(polymers_propensities_unnorm.values())

    polymers_propensities = dict(map(
        lambda x: (x[0], x[1] / total),
        polymers_propensities_unnorm.items()
    ))

    sector_polymers[get_sector_key(region, sector)] = polymers_propensities

In [18]:
def get_sector_polymer_propensity(region, sector, polymer):
    return sector_polymers[get_sector_key(region, sector)][polymer]

<br>

## Perform calculations

With the aforementioned consumption ($C$) approximation in mind, we use our now indexed data to estimate masses for each polymer $p$ in a region $r$ per sector $s$. This is the volume $v$ that saw the fate (such as incineration) $f$ in year $y$ as formalized below where volume is a function:

$v(r, y, s, p, f) = W_{r,y} * \frac{W_{f}}{W_{total}} * \frac{C_{s}}{C_{total}} * \frac{m_{p}}{m_{s}}$

This effectively determines the following in order:

 - Total waste for a region / year combination.
 - Of the total waste, the waste going to the target end of life fate like landfill.
 - Of the fate waste, the amount from a given sector like construction.
 - Of the waste from the sector, the amount coming from a given polymer like LDPE.

Finally, these estimations are summed across sectors.

### Calculate at the sector level

We start by approximating the amount of each polymer in each sector going to each waste fate within a region / year combination like China 2040.

In [19]:
def get_estimated_waste(region, year, sector, polymer, fate):
    total_waste = get_total_waste(region, year)
    
    percent_in_fate = get_fate_propensity(region, year, fate)
    waste_in_fate = total_waste * percent_in_fate
    
    percent_from_sector = get_sector_propensity(region, year, sector)
    waste_in_fate_from_sector = waste_in_fate * percent_from_sector
    
    percent_from_polymer = get_sector_polymer_propensity(region, sector, polymer)
    waste_polymer_in_fate_from_sector = percent_from_polymer * waste_in_fate_from_sector
    
    return waste_polymer_in_fate_from_sector

In [20]:
combinations_tuples = itertools.product(
    REGIONS,
    YEARS,
    SECTORS,
    POLYMERS,
    FATES
)

combinations = map(lambda x: {
    'region': x[0],
    'year': x[1],
    'sector': x[2],
    'polymer': x[3],
    'fate': x[4]
}, combinations_tuples)

combinations_with_vals = map(lambda x: {
    'region': x['region'],
    'year': x['year'],
    'sector': x['sector'],
    'polymer': x['polymer'],
    'fate': x['fate'],
    'group': '\t'.join(map(lambda x: str(x), [
        x['region'],
        x['year'],
        x['polymer'],
        x['fate']
    ])),
    'amount': get_estimated_waste(
        x['region'],
        x['year'],
        x['sector'],
        x['polymer'],
        x['fate']
    )
}, combinations)

### Summarize across sectors

We then sum across all sectors. For example, there will be four rows for Majority World in 2035: one for recycling, one for landfill, one for incineration, and one for mismanaged.

In [21]:
individual_tuples = list(map(lambda x: (x['group'], x['amount']), combinations_with_vals))
reduced_tuples = toolz.itertoolz.reduceby(
    lambda x: x[0], lambda a, b: (a[0], a[1] + b[1]),
    individual_tuples
).values()
reduced_thin_dicts = list(map(lambda x: {'group': x[0], 'amount': x[1]}, reduced_tuples))

<br>

## Output

Having generated this summarization, we write out the results to a CSV file.

In [22]:
def make_output_record(target_thin_dict):
    pieces = target_thin_dict['group'].split('\t')
    return {
        'region': pieces[0],
        'year': pieces[1],
        'polymer': pieces[2],
        'fate': pieces[3],
        'approximateAmount': target_thin_dict['amount']
    }

In [23]:
output_records = map(make_output_record, reduced_thin_dicts)

with open('polymer_eol_approximate.csv', 'w') as f:
    writer = csv.DictWriter(f, fieldnames=['region', 'year', 'polymer', 'fate', 'approximateAmount'])
    writer.writeheader()
    writer.writerows(output_records)

<br>

## Discussion

Note that this method introduces a significant approximation. Due to lifecycle distributions, the sector ratios at time of waste generation may not be the same as if one were to project through time to get the prior sector ratios which generated the waste in question. Additionally, we do not have sector or product-specific EOL fate propensitives. Using proportionality absent additional information, this introduces further uncertainty into these estimates. Even so, this use of "end of life year ratios" may provide a sufficient estimation for many use cases in which these approximations are acceptable.